In [1]:
import pandas as pd

pd.set_option('display.max_columns', 10000)

In [2]:
FUNCTIONALLY_OMITTED_STATES = {
    "Alabama",
    "Florida",
}

# Cleanup and Munge Data

In [3]:
df_all_murder = pd.read_excel('data/table-4.xls', engine='xlrd', skiprows=3, skipfooter=9)
df_homicide = pd.read_excel('data/table-20.xls', engine='xlrd', skiprows=3, skipfooter=5)

df_owners = pd.read_excel('data/TL-354-State-Level Estimates of Household Firearm Ownership.xlsx', engine='xlrd', sheet_name='State-Level Data & Factor Score')

df_population = pd.read_excel('data/nst-est2019-01.xlsx', engine='xlrd', skiprows=3, skipfooter=7)

In [4]:
df_all_murder_clean = (
    df_all_murder
    .drop(0, axis=0)
    .reset_index(drop=True)
    .drop(df_all_murder.columns[[-3, -2, -1]], axis=1)
)

# fill empty cells in Area column
df_all_murder_clean['Area'] = df_all_murder_clean.Area.ffill()

# remove footnotes and newlines in column names
df_all_murder_clean.columns = [
    c
    .replace('\n', '')
    .replace('2', ' ')
    .replace('3', ' ')
    .replace('4', ' ')
    for c in df_all_murder_clean.columns
]

# properly label the per-100k columns that are immediately right of their raw count source data 
left_shift_columns = list(df_all_murder_clean.columns)[:-1]
left_shift_columns.insert(0, left_shift_columns[-1])
clean_columns = [
    f'{prev_column_name} per 100k' if 'Unnamed' in column_name else column_name
    for column_name, prev_column_name
    in list(zip(df_all_murder_clean.columns, left_shift_columns))
]
df_all_murder_clean.columns = clean_columns

# remove footnotes from Area column
df_all_murder_clean['Area'] = df_all_murder_clean.Area.map(
    lambda s: s
        .replace('1', '')
        .replace('2', '')
        .replace('3', '')
        .replace('4', '')
        .replace('5', '')
        .replace('6', '')
        .replace('7', '')
        .replace(',', '')
        .strip()
)

# select solely 2019 (dropping rows for Year == 2018 and Year == percent change)
df_all_murder_clean = df_all_murder_clean[df_all_murder_clean.Year.isin([2019])].reset_index(drop=True)

df_all_murder_clean

,Area,Year,Population,Violent crime,Violent crime per 100k,Murder and nonnegligent manslaughter,Murder and nonnegligent manslaughter per 100k,Rape(revised definition),Rape(revised definition) per 100k,Robbery,Robbery per 100k,Aggravated assault,Aggravated assault per 100k,Property crime,Property crime per 100k,Burglary,Burglary per 100k,Larceny-theft,Larceny-theft per 100k,Motor vehicle theft,Motor vehicle theft per 100k
0,United States Total,2019,328239523,1245410,379.4,16425.0,5,139815.0,42.6,267988.0,81.6,821182.0,250.2,6925677.0,2109.9,1117696.0,340.5,5086096.0,1549.5,721885.0,219.9
1,Northeast,2019,55982803,163717,292.4,1834.0,3.3,17315.0,30.9,40073.0,71.6,104495.0,186.7,755968.0,1350.4,93798.0,167.5,611369.0,1092.1,50801.0,90.7
2,New England,2019,14845063,36350,244.9,345.0,2.3,4850.0,32.7,6532.0,44,24623.0,165.9,190511.0,1283.3,26445.0,178.1,148695.0,1001.6,15371.0,103.5
3,Connecticut,2019,3565287,6546,183.6,104.0,2.9,771.0,21.6,1929.0,54.1,3742.0,105,50862.0,1426.6,6441.0,180.7,38457.0,1078.7,5964.0,167.3
4,Maine,2019,1344212,1548,115.2,20.0,1.5,516.0,38.4,188.0,14,824.0,61.3,16743.0,1245.6,2350.0,174.8,13667.0,1016.7,726.0,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,California,2019,39512223,174331,441.2,1690.0,4.3,14799.0,37.5,52301.0,132.4,105541.0,267.1,921114.0,2331.2,152555.0,386.1,626802.0,1586.3,141757.0,358.8
62,Hawaii,2019,1415872,4042,285.5,48.0,3.4,765.0,54,1131.0,79.9,2098.0,148.2,40228.0,2841.2,5340.0,377.2,29634.0,2093,5254.0,371.1
63,Oregon,2019,4217737,11995,284.4,116.0,2.8,1778.0,42.2,2276.0,54,7825.0,185.5,115170.0,2730.6,14724.0,349.1,85261.0,2021.5,15185.0,360
64,Washington,2019,7614893,22377,293.9,198.0,2.6,3332.0,43.8,5147.0,67.6,13700.0,179.9,204224.0,2681.9,34540.0,453.6,145282.0,1907.9,24402.0,320.5


In [5]:
df_homicide_clean = df_homicide

# remove newlines in column names
df_homicide_clean.columns = [c.replace('\n', ' ') for c in df_homicide_clean.columns]

# rename Area column (and remove footnotes from column names)
df_homicide_clean = df_homicide_clean.rename({
    'State': 'Area',
    'Total murders1': 'Total murders',
    'Hands, fists, feet, etc.2': 'Hands, fists, feet, etc.',
}, axis='columns')

# remove footnotes from Area column entries
df_homicide_clean['Area'] = df_homicide_clean.Area.map(lambda s: s.replace('3', '').replace('4', '').replace('5', ''))

df_homicide_clean

,Area,Total murders,Total firearms,Handguns,Rifles,Shotguns,Firearms (type unknown),Knives or cutting instruments,Other weapons,"Hands, fists, feet, etc."
0,Alabama,4,3,3,0,0,0,0,1,0
1,Alaska,69,44,17,1,6,20,8,5,12
2,Arizona,337,213,170,6,6,31,47,67,10
3,Arkansas,231,177,87,10,5,75,21,29,4
4,California,1679,1142,762,34,26,320,252,183,102
5,Colorado,209,135,83,5,3,44,32,30,12
6,Connecticut,104,65,11,1,0,53,15,17,7
7,Delaware,48,40,13,0,0,27,2,3,3
8,District of Columbia,166,136,52,0,0,84,18,2,10
9,Florida,1,0,0,0,0,0,0,1,0


In [6]:
# remove functionally worthless values from Alabama and Florida
alabama_indices = df_homicide_clean.index[df_homicide_clean.Area == 'Alabama'].tolist()
florida_indices = df_homicide_clean.index[df_homicide_clean.Area == 'Florida'].tolist()
df_homicide_clean.loc[alabama_indices] = ['Alabama'] + [float('nan')] * (len(df_homicide_clean.columns) - 1)
df_homicide_clean.loc[florida_indices] = ['Florida'] + [float('nan')] * (len(df_homicide_clean.columns) - 1)

df_homicide_clean

,Area,Total murders,Total firearms,Handguns,Rifles,Shotguns,Firearms (type unknown),Knives or cutting instruments,Other weapons,"Hands, fists, feet, etc."
0,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,69.0,44.0,17.0,1.0,6.0,20.0,8.0,5.0,12.0
2,Arizona,337.0,213.0,170.0,6.0,6.0,31.0,47.0,67.0,10.0
3,Arkansas,231.0,177.0,87.0,10.0,5.0,75.0,21.0,29.0,4.0
4,California,1679.0,1142.0,762.0,34.0,26.0,320.0,252.0,183.0,102.0
5,Colorado,209.0,135.0,83.0,5.0,3.0,44.0,32.0,30.0,12.0
6,Connecticut,104.0,65.0,11.0,1.0,0.0,53.0,15.0,17.0,7.0
7,Delaware,48.0,40.0,13.0,0.0,0.0,27.0,2.0,3.0,3.0
8,District of Columbia,166.0,136.0,52.0,0.0,0.0,84.0,18.0,2.0,10.0
9,Florida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# drop all rows except most recent Year == '2016'
df_owners_clean = df_owners[df_owners.Year.isin(['2016'])].reset_index(drop=True)

# rename Area column
df_owners_clean = df_owners_clean.rename({'STATE': 'Area'}, axis='columns')

df_owners_clean

,FIP,Year,Area,HFR,HFR_se,universl,permit,Fem_FS_S,Male_FS_S,BRFSS,GALLUP,GSS,PEW,HuntLic,GunsAmmo,BackChk,PewQChng,BS1,BS2,BS3
0,1,2016,Alabama,0.528,0.028,0,0,0.646341,0.711538,-9.0,-9.0,0.471448,0.649361,0.341758,-0.219699,0.270898,1,12.0,12.0,12.0
1,2,2016,Alaska,0.572,0.032,0,0,0.369565,0.653061,-9.0,-9.0,-9.000000,0.666125,0.657791,0.740572,1.937272,1,12.0,12.0,12.0
2,4,2016,Arizona,0.360,0.028,0,0,0.409091,0.647716,-9.0,-9.0,0.372834,0.387098,0.243218,0.071988,-0.447685,1,12.0,12.0,12.0
3,5,2016,Arkansas,0.518,0.028,0,0,0.510638,0.625604,-9.0,-9.0,0.480983,0.661608,0.363895,0.093631,0.308392,1,12.0,12.0,12.0
4,6,2016,California,0.163,0.028,1,1,0.182548,0.428571,-9.0,-9.0,0.186860,0.240008,0.160671,-1.473130,-0.898778,1,12.0,12.0,12.0
5,8,2016,Colorado,0.379,0.028,1,0,0.337165,0.578831,-9.0,-9.0,0.322770,0.476516,0.281931,0.208845,0.814156,1,12.0,12.0,12.0
6,9,2016,Connecticut,0.188,0.028,1,1,0.078261,0.358156,-9.0,-9.0,0.245675,0.272317,0.188097,-0.843628,-0.439763,1,12.0,12.0,12.0
7,10,2016,Delaware,0.387,0.028,1,0,0.428571,0.571429,-9.0,-9.0,0.381304,0.499979,0.231977,-0.676874,-0.209439,1,12.0,12.0,12.0
8,12,2016,Florida,0.288,0.028,0,0,0.353690,0.591430,-9.0,-9.0,0.232072,0.374913,0.127689,-0.777549,-0.222022,1,12.0,12.0,12.0
9,13,2016,Georgia,0.377,0.028,0,0,0.472669,0.659381,-9.0,-9.0,0.310040,0.472258,0.354964,-0.916168,-0.801952,1,12.0,12.0,12.0


In [8]:
# rename Area column
df_population_clean = df_population.rename({'Unnamed: 0': 'Area'}, axis='columns')

# remove trailing periods
df_population_clean['Area'] = df_population_clean.Area.map(lambda r: r.replace('.', ''))

df_population_clean

,Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538,308758105,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
1,Northeast,55317240,55318443,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803
2,Midwest,66927001,66929725,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004
3,South,114555744,114563030,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448
4,West,71945553,71946907,72100436,72788329,73477823,74167130,74925793,75742555,76559681,77257329,77834820,78347268
5,Alabama,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
6,Alaska,710231,710249,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
7,Arizona,6392017,6392288,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
8,Arkansas,2915918,2916031,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
9,California,37253956,37254519,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223


In [9]:
df = (
    df_homicide_clean
        .merge(df_all_murder_clean, on='Area')
        .merge(df_owners_clean, on='Area')
        .merge(df_population_clean, on='Area')
)
df

,Area,Total murders,Total firearms,Handguns,Rifles,Shotguns,Firearms (type unknown),Knives or cutting instruments,Other weapons,"Hands, fists, feet, etc.",Year_x,Population,Violent crime,Violent crime per 100k,Murder and nonnegligent manslaughter,Murder and nonnegligent manslaughter per 100k,Rape(revised definition),Rape(revised definition) per 100k,Robbery,Robbery per 100k,Aggravated assault,Aggravated assault per 100k,Property crime,Property crime per 100k,Burglary,Burglary per 100k,Larceny-theft,Larceny-theft per 100k,Motor vehicle theft,Motor vehicle theft per 100k,FIP,Year_y,HFR,HFR_se,universl,permit,Fem_FS_S,Male_FS_S,BRFSS,GALLUP,GSS,PEW,HuntLic,GunsAmmo,BackChk,PewQChng,BS1,BS2,BS3,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,4903185,25046,510.8,358.0,7.3,2068.0,42.2,3941.0,80.4,18679.0,381,131133.0,2674.4,26079.0,531.9,92477.0,1886.1,12577.0,256.5,1,2016,0.528,0.028,0,0,0.646341,0.711538,-9.0,-9.0,0.471448,0.649361,0.341758,-0.219699,0.270898,1,12.0,12.0,12.0,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,Alaska,69.0,44.0,17.0,1.0,6.0,20.0,8.0,5.0,12.0,2019,731545,6343,867.1,69.0,9.4,1088.0,148.7,826.0,112.9,4360.0,596,21294.0,2910.8,3563.0,487.1,15114.0,2066,2617.0,357.7,2,2016,0.572,0.032,0,0,0.369565,0.653061,-9.0,-9.0,-9.000000,0.666125,0.657791,0.740572,1.937272,1,12.0,12.0,12.0,710231,710249,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
2,Arizona,337.0,213.0,170.0,6.0,6.0,31.0,47.0,67.0,10.0,2019,7278717,33141,455.3,365.0,5,3662.0,50.3,6410.0,88.1,22704.0,311.9,177638.0,2440.5,28699.0,394.3,130788.0,1796.9,18151.0,249.4,4,2016,0.360,0.028,0,0,0.409091,0.647716,-9.0,-9.0,0.372834,0.387098,0.243218,0.071988,-0.447685,1,12.0,12.0,12.0,6392017,6392288,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
3,Arkansas,231.0,177.0,87.0,10.0,5.0,75.0,21.0,29.0,4.0,2019,3017804,17643,584.6,242.0,8,2331.0,77.2,1557.0,51.6,13513.0,447.8,86250.0,2858,18095.0,599.6,60735.0,2012.6,7420.0,245.9,5,2016,0.518,0.028,0,0,0.510638,0.625604,-9.0,-9.0,0.480983,0.661608,0.363895,0.093631,0.308392,1,12.0,12.0,12.0,2915918,2916031,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
4,California,1679.0,1142.0,762.0,34.0,26.0,320.0,252.0,183.0,102.0,2019,39512223,174331,441.2,1690.0,4.3,14799.0,37.5,52301.0,132.4,105541.0,267.1,921114.0,2331.2,152555.0,386.1,626802.0,1586.3,141757.0,358.8,6,2016,0.163,0.028,1,1,0.182548,0.428571,-9.0,-9.0,0.186860,0.240008,0.160671,-1.473130,-0.898778,1,12.0,12.0,12.0,37253956,37254519,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223
5,Colorado,209.0,135.0,83.0,5.0,3.0,44.0,32.0,30.0,12.0,2019,5758736,21938,381,218.0,3.8,3872.0,67.2,3663.0,63.6,14185.0,246.3,149189.0,2590.7,20064.0,348.4,107012.0,1858.3,22113.0,384,8,2016,0.379,0.028,1,0,0.337165,0.578831,-9.0,-9.0,0.322770,0.476516,0.281931,0.208845,0.814156,1,12.0,12.0,12.0,5029196,5029319,5047349,5121108,5192647,5269035,5350101,5450623,5539215,5611885,5691287,5758736
6,Connecticut,104.0,65.0,11.0,1.0,0.0,53.0,15.0,17.0,7.0,2019,3565287,6546,183.6,104.0,2.9,771.0,21.6,1929.0,54.1,3742.0,105,50862.0,1426.6,6441.0,180.7,38457.0,1078.7,5964.0,167.3,9,2016,0.188,0.028,1,1,0.078261,0.358156,-9.0,-9.0,0.245675,0.272317,0.188097,-0.843628,-0.439763,1,12.0,12.0,12.0,3574097,3574147,3579114,3588283,3594547,3594841,3594524,3587122,3578141,3573297,3571520,3565287
7,Delaware,48.0,40.0,13.0,0.0,0.0,27.0,2.0,3.0,3.0,2019,973764,4115,422.6,48.0,4.9,310.0,31.8,790.0,81.1,2967.0,304.7,21931.0,2252.2,2968.0,304.8,17359.0,1782.7,1604.0,164.7,10,2016,0.387,0.028,1,0,0.428571,0.571429,-9.0,-9.0,0.381304,0.499979,0.231977,-0.676874,-0.209439,1,12.0,12.0,12.0,897934,897937,899593,907381,915179,923576,932487,941252,948921,956823,965479,973764
8,Florida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,21477737,81270,378.4,1122.0,5.2,8456.0,39

In [10]:
df['MANUAL Murder and nonnegligent manslaughter per 100k'] = df['Murder and nonnegligent manslaughter'] * 100000 / df[2019]
df['Murder rate per 100k'] = df['Total murders'] * 100000 / df[2019]
df['Gun murder rate per 100k'] = df['Total firearms'] * 100000 / df[2019]
df['HFR'] = df['HFR']*100

# Joined Table of 2019 Data


In [11]:
df[[
    'Area',
    2019,
    'Murder and nonnegligent manslaughter',
    'Murder and nonnegligent manslaughter per 100k',
    'MANUAL Murder and nonnegligent manslaughter per 100k',
    'Total murders',
    'Total firearms',
    'HFR',
    'Murder rate per 100k',
    'Gun murder rate per 100k',
]]

,Area,2019,Murder and nonnegligent manslaughter,Murder and nonnegligent manslaughter per 100k,MANUAL Murder and nonnegligent manslaughter per 100k,Total murders,Total firearms,HFR,Murder rate per 100k,Gun murder rate per 100k
0,Alabama,4903185,358.0,7.3,7.301377,NaN,NaN,52.8,NaN,NaN
1,Alaska,731545,69.0,9.4,9.432092,69.0,44.0,57.2,9.432092,6.014668
2,Arizona,7278717,365.0,5,5.014620,337.0,213.0,36.0,4.629937,2.926340
3,Arkansas,3017804,242.0,8,8.019076,231.0,177.0,51.8,7.654573,5.865192
4,California,39512223,1690.0,4.3,4.277157,1679.0,1142.0,16.3,4.249318,2.890245
5,Colorado,5758736,218.0,3.8,3.785553,209.0,135.0,37.9,3.629269,2.344264
6,Connecticut,3565287,104.0,2.9,2.917016,104.0,65.0,18.8,2.917016,1.823135
7,Delaware,973764,48.0,4.9,4.929326,48.0,40.0,38.7,4.929326,4.107771
8,Florida,21477737,1122.0,5.2,5.224014,NaN,NaN,28.8,NaN,NaN
9,Georgia,10617423,654.0,6.2,6.159687,445.0,367.0,37.7,4.191224,3.456583


In [12]:
EPSILON = 0.05

# sanity check that manually calculated per 100k value of
# "murder and nonnegligent manslughter" matches the table-4 provided rate
rates_differ = (
    df['MANUAL Murder and nonnegligent manslaughter per 100k']
    - df['Murder and nonnegligent manslaughter per 100k']
).abs().gt(EPSILON)

assert not rates_differ.any()

# Produce Wikitext Output

In [13]:
QUARTILE_LOOKUP = {
    "1": "#ccffcc",
    "2": "#ffffcc",
    "3": "#ffddcc",
    "4": "#ffbbcc",
}

# columns we need to quantize into quartiles for background colors
TABLE_COLUMNS = [
    2019,
    'Murder and nonnegligent manslaughter',
    'Total murders',
    'Total firearms',
    'HFR',
    'MANUAL Murder and nonnegligent manslaughter per 100k',
    'Murder rate per 100k',
    'Gun murder rate per 100k',
]

TABLE20_FOOTNOTE_STATE_LOOKUP = {
    "Alabama": "Limited data for 2019 were available for Alabama.",
    "Florida": "No data available.", # "Data submitted through the Bureau of Indian Affairs.",
    "Illinois": "Limited supplemental homicide data were received.",
}

TABLE20_FOOTNOTE_COLUMN_LOOKUP = {
    'Total murders',
    'Total firearms',
    'Murder rate per 100k',
    'Gun murder rate per 100k',
}


In [14]:
# 25th, 50th, and 75th percentiles for each column
table_column_quartiles = {
    c: df[c].quantile([0.25, 0.50, 0.75]).tolist()
    for c in TABLE_COLUMNS
}

table_column_quartiles

{2019: [1827712.25, 4558233.5, 7530849.0],
 'Murder and nonnegligent manslaughter': [62.25, 208.0, 528.0],
 'Total murders': [55.5, 183.5, 458.25],
 'Total firearms': [39.0, 127.0, 370.0],
 'HFR': [35.349999999999994, 39.95, 52.175000000000004],
 'MANUAL Murder and nonnegligent manslaughter per 100k': [2.5447291673366483,
  4.47745000779638,
  6.126236613331899],
 'Murder rate per 100k': [2.351508171913852,
  3.843225800246702,
  5.131281516061836],
 'Gun murder rate per 100k': [1.4843459028100705,
  2.713165624757706,
  3.9479265709991926]}

In [15]:
def get_background_color_for_column_value(column, value):
    if value == '-':
        return None
    assert column in table_column_quartiles
    q25, q50, q75 = table_column_quartiles[column]
    if value < q25:
        return QUARTILE_LOOKUP["1"]
    if value < q50:
        return QUARTILE_LOOKUP["2"]
    if value < q75:
        return QUARTILE_LOOKUP["3"]
    else:
        return QUARTILE_LOOKUP["4"]

In [16]:
# final roundings for data presentation
df['Murder and nonnegligent manslaughter'] = df['Murder and nonnegligent manslaughter'].astype(int)
df['HFR'] = df['HFR'].astype(int)
df['Murder and nonnegligent manslaughter per 100k'] = df['Murder and nonnegligent manslaughter per 100k'].astype(float).round(1)
df['MANUAL Murder and nonnegligent manslaughter per 100k'] = df['MANUAL Murder and nonnegligent manslaughter per 100k'].round(1)

# these values need special treatment because they have '-' marks for excluded data
# (for Alabama and Florida) as a result of table20 footnotes
df['Total murders'] = df['Total murders'].fillna('-').map(lambda value: value if value == '-' else int(value))
df['Total firearms'] = df['Total firearms'].fillna('-').map(lambda value: value if value == '-' else int(value))
df['Murder rate per 100k'] = df['Murder rate per 100k'].fillna('-').map(lambda value: value if value == '-' else round(value, 1))
df['Gun murder rate per 100k'] = df['Gun murder rate per 100k'].fillna('-').map(lambda value: value if value == '-' else round(value, 1))
df

,Area,Total murders,Total firearms,Handguns,Rifles,Shotguns,Firearms (type unknown),Knives or cutting instruments,Other weapons,"Hands, fists, feet, etc.",Year_x,Population,Violent crime,Violent crime per 100k,Murder and nonnegligent manslaughter,Murder and nonnegligent manslaughter per 100k,Rape(revised definition),Rape(revised definition) per 100k,Robbery,Robbery per 100k,Aggravated assault,Aggravated assault per 100k,Property crime,Property crime per 100k,Burglary,Burglary per 100k,Larceny-theft,Larceny-theft per 100k,Motor vehicle theft,Motor vehicle theft per 100k,FIP,Year_y,HFR,HFR_se,universl,permit,Fem_FS_S,Male_FS_S,BRFSS,GALLUP,GSS,PEW,HuntLic,GunsAmmo,BackChk,PewQChng,BS1,BS2,BS3,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,MANUAL Murder and nonnegligent manslaughter per 100k,Murder rate per 100k,Gun murder rate per 100k
0,Alabama,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,4903185,25046,510.8,358,7.3,2068.0,42.2,3941.0,80.4,18679.0,381,131133.0,2674.4,26079.0,531.9,92477.0,1886.1,12577.0,256.5,1,2016,52,0.028,0,0,0.646341,0.711538,-9.0,-9.0,0.471448,0.649361,0.341758,-0.219699,0.270898,1,12.0,12.0,12.0,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,7.3,-,-
1,Alaska,69,44,17.0,1.0,6.0,20.0,8.0,5.0,12.0,2019,731545,6343,867.1,69,9.4,1088.0,148.7,826.0,112.9,4360.0,596,21294.0,2910.8,3563.0,487.1,15114.0,2066,2617.0,357.7,2,2016,57,0.032,0,0,0.369565,0.653061,-9.0,-9.0,-9.000000,0.666125,0.657791,0.740572,1.937272,1,12.0,12.0,12.0,710231,710249,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,9.4,9.4,6
2,Arizona,337,213,170.0,6.0,6.0,31.0,47.0,67.0,10.0,2019,7278717,33141,455.3,365,5.0,3662.0,50.3,6410.0,88.1,22704.0,311.9,177638.0,2440.5,28699.0,394.3,130788.0,1796.9,18151.0,249.4,4,2016,36,0.028,0,0,0.409091,0.647716,-9.0,-9.0,0.372834,0.387098,0.243218,0.071988,-0.447685,1,12.0,12.0,12.0,6392017,6392288,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,5.0,4.6,2.9
3,Arkansas,231,177,87.0,10.0,5.0,75.0,21.0,29.0,4.0,2019,3017804,17643,584.6,242,8.0,2331.0,77.2,1557.0,51.6,13513.0,447.8,86250.0,2858,18095.0,599.6,60735.0,2012.6,7420.0,245.9,5,2016,51,0.028,0,0,0.510638,0.625604,-9.0,-9.0,0.480983,0.661608,0.363895,0.093631,0.308392,1,12.0,12.0,12.0,2915918,2916031,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,8.0,7.7,5.9
4,California,1679,1142,762.0,34.0,26.0,320.0,252.0,183.0,102.0,2019,39512223,174331,441.2,1690,4.3,14799.0,37.5,52301.0,132.4,105541.0,267.1,921114.0,2331.2,152555.0,386.1,626802.0,1586.3,141757.0,358.8,6,2016,16,0.028,1,1,0.182548,0.428571,-9.0,-9.0,0.186860,0.240008,0.160671,-1.473130,-0.898778,1,12.0,12.0,12.0,37253956,37254519,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,4.3,4.2,2.9
5,Colorado,209,135,83.0,5.0,3.0,44.0,32.0,30.0,12.0,2019,5758736,21938,381,218,3.8,3872.0,67.2,3663.0,63.6,14185.0,246.3,149189.0,2590.7,20064.0,348.4,107012.0,1858.3,22113.0,384,8,2016,37,0.028,1,0,0.337165,0.578831,-9.0,-9.0,0.322770,0.476516,0.281931,0.208845,0.814156,1,12.0,12.0,12.0,5029196,5029319,5047349,5121108,5192647,5269035,5350101,5450623,5539215,5611885,5691287,5758736,3.8,3.6,2.3
6,Connecticut,104,65,11.0,1.0,0.0,53.0,15.0,17.0,7.0,2019,3565287,6546,183.6,104,2.9,771.0,21.6,1929.0,54.1,3742.0,105,50862.0,1426.6,6441.0,180.7,38457.0,1078.7,5964.0,167.3,9,2016,18,0.028,1,1,0.078261,0.358156,-9.0,-9.0,0.245675,0.272317,0.188097,-0.843628,-0.439763,1,12.0,12.0,12.0,3574097,3574147,3579114,3588283,3594547,3594841,3594524,3587122,3578141,3573297,3571520,3565287,2.9,2.9,1.8
7,Delaware,48,40,13.0,0.0,0.0,27.0,2.0,3.0,3.0,2019,973764,4115,422.6,48,4.9,310.0,31.8,790.0,81.1,2967.0,304.7,21931.0,2252.2,2968.0,304.8,17359.0,1782.7,1604.0,164.7,10,2016,38,0.028,1,0,0.428571,0.571429,-9.0,-9.0,0.381304,0.499979,0.231977,-0.676874,-0.209439,1,12.0,12.0,12.0,897934,897937,899593,907381,915179,923576,932487,941252,948921,9568

In [17]:
# sample output formatting per each state's row:

#|-
#| [[Alabama]]
#| style="background: #ffddcc" | 4,853,875
#| style="background: #ffddcc" | 348
#| —{{efn|name=limited-data|Limited supplemental homicide data were received.}}
#| —{{efn|name=limited-data}}
#| style="background: #ffbbcc" | 48.9
#| style="background: #ffbbcc" | 7.2
#| —{{efn|name=limited-data}}
#| —{{efn|name=limited-data}}
#|-
#| [[Alaska]]
#| style="background: #ccffcc" | 737,709
#| style="background: #ccffcc" | 59
#| style="background: #ccffcc" | 57
#| style="background: #ffffcc" | 39
#| style="background: #ffbbcc" | 61.7
#| style="background: #ffbbcc" | 8.0
#| style="background: #ffbbcc" | 7.7
#| style="background: #ffbbcc" | 5.3


# desired row cells are:
# ----------------
# State
# Population
# Murder and Nonnegligent Manslaughters
# Murders (of all murders with supplemental homicide data)
# Gun Murders (count)
# Household Firearm Ownership
# Murder and Nonnegligent Manslaughter Rate
# Murder Rate
# Firearm Murder Rate


for i, series in df.iterrows():
    table20_footnoted_state = series['Area'] in TABLE20_FOOTNOTE_STATE_LOOKUP
    print(f'|-')
    print(f'| [[{series.Area}]]')
    for c in TABLE_COLUMNS:
        # if it's a footnoted area in table20, and this is a table20 column
        if table20_footnoted_state and c in TABLE20_FOOTNOTE_COLUMN_LOOKUP:
            cell_string = '| '
            cell_color = get_background_color_for_column_value(c, series[c])
            if cell_color:
                 cell_string += f'style="background: {cell_color}" | '
            cell_string += f'{series[c]}'
            
            # only define the full efn on the first cell, call by reference after
            if c == 'Total murders':
                cell_string += '{{' + f'efn|name={series.Area}-limited-data|{TABLE20_FOOTNOTE_STATE_LOOKUP[series.Area]}' + '}}'
            else:
                cell_string += '{{' + f'efn|name={series.Area}-limited-data' + '}}'
            print(cell_string)
        else:
            print(f'| style="background: {get_background_color_for_column_value(c, series[c])}" | {series[c]}')


|-
| [[Alabama]]
| style="background: #ffddcc" | 4903185
| style="background: #ffddcc" | 358
| -{{efn|name=Alabama-limited-data|Limited data for 2019 were available for Alabama.}}
| -{{efn|name=Alabama-limited-data}}
| style="background: #ffddcc" | 52
| style="background: #ffbbcc" | 7.3
| -{{efn|name=Alabama-limited-data}}
| -{{efn|name=Alabama-limited-data}}
|-
| [[Alaska]]
| style="background: #ccffcc" | 731545
| style="background: #ffffcc" | 69
| style="background: #ffffcc" | 69
| style="background: #ffffcc" | 44
| style="background: #ffbbcc" | 57
| style="background: #ffbbcc" | 9.4
| style="background: #ffbbcc" | 9.4
| style="background: #ffbbcc" | 6.0
|-
| [[Arizona]]
| style="background: #ffddcc" | 7278717
| style="background: #ffddcc" | 365
| style="background: #ffddcc" | 337
| style="background: #ffddcc" | 213
| style="background: #ffffcc" | 36
| style="background: #ffddcc" | 5.0
| style="background: #ffddcc" | 4.6
| style="background: #ffddcc" | 2.9
|-
| [[Arkansas]]
| style="